In [1]:
import itertools

from sympy.polys.polytools import degree, div, gcd, refine_root, sturm
from sympy import Float, LC, LT, Matrix, Poly, diff, sign, symbols
from mpmath import ceil, root
from sympy.polys.subresultants_qq_zz import euclid_amv

In [2]:
x = symbols("x")

# Πρώτος αλγόριθμος υπολογισμού των prs με τριγωνοποίηση πινάκων

In [3]:
# function to create a polynomial from given coefficients
def create_polynomial_from_coeff(coeffs, x=None):
    if x is None:
        x = symbols('x')
    pol = sum(coeff * x**i for i, coeff in enumerate(reversed(coeffs)))
    return pol


# calculates delta from Anna Johnson paper
def delta_f(r1, r2, x):
    return degree(r1, x) - degree(r2, x) + 1


# calculates psi from Anna Johnson paper
def psi_f(r1, delta_prev, psi_prev, x):
    return ((-LC(r1, x))**(delta_prev - 1)) / (psi_prev**(delta_prev - 2))


# calculates beta from Anna Johnson paper
def beta_f(r1, delta_prev, psi, x):
    return (psi**(delta_prev - 1)) * (-LC(r1, x))

In [4]:
# this function is mimicking the pivot function from Xcas
# Although it only uses a single argument m which is the matrix it has the same functionality as
# It triangulates the matrix m it a loop and each iteration is like calling the pivot from Xcas
# meaning it does not effect the rows above the current row
def pivot(m):
    rows, cols = m.shape
    for iteration in range(rows - 1):
        # m[iteration, iteration] is actually the pivot, the first nonzero element it a row
        if m[iteration, iteration] == 0:
            continue

        # after checking that the pivot is not zero,
        # check each number under the pivot and make it zero
        for row in range(iteration + 1, rows):
            if m[row, iteration] == 0:
                continue
            m[row, :] = m[row, :] * m[iteration, iteration] - m[row, iteration] * m[iteration, :]
    return m



# We had implemented the division with the betas prior to your announcement
# that it wasn't required to do it so we added a flag to disable them
def euclid_triang(f, g, x, use_beta=True):
    prs = [f, g]
    r_1 = f
    r0 = g
    delta = delta_f(r_1, r0, x)
    r_1_coeffs = Poly(r_1).coeffs()  # r-1
    r0_coeffs = Poly(r0).coeffs()  # r0

    m = Matrix([[*r0_coeffs, 0], [0, *r0_coeffs], r_1_coeffs])
    m = pivot(m)
    r = create_polynomial_from_coeff(m[-1, :], x)

    if not use_beta:
        r_1 = r0
        r0 = r
        while True:
            r_1_coeffs = Poly(r_1).coeffs()  # r-1
            r0_coeffs = Poly(r0).coeffs()  # r0
            m = Matrix([[*r0_coeffs, 0], [0, *r0_coeffs], r_1_coeffs])
            m = pivot(m)
            r = create_polynomial_from_coeff(m[-1, :])
            prs.append(r)
            if degree(r, x) <= 0:
                break

            r_1 = r0
            r0 = r
        return prs
    else:

        i = 0
        psis = [-1]
        betas = [psis[i]**delta]
        deltas = [delta]

        r = r / betas[i]
        prs.append(r)

        r_1 = r0
        r0 = r

        i = i + 1
        while True:

            delta = delta_f(r_1, r0, x)
            r_1_coeffs = Poly(r_1).coeffs()  # r-1
            r0_coeffs = Poly(r0).coeffs()  # r0
            deltas.append(delta)

            m = Matrix([[*r0_coeffs, 0], [0, *r0_coeffs], r_1_coeffs])
            m = pivot(m)
            r = create_polynomial_from_coeff(m[-1, :])

            psi = psi_f(r_1, deltas[i - 1], psis[i - 1], x)
            psis.append(psi)

            beta = beta_f(r_1, deltas[i - 1], psis[i], x)
            betas.append(beta)

            r = r / abs(betas[i])
            prs.append(r)

            if degree(r, x) <= 0:
                break

            i = i + 1
            r_1 = r0
            r0 = r

        return prs

In [5]:
f1 = x**3 + 3 * x**2 - 7 * x + 7
g1 = 3 * x**2 + 6 * x - 7
f2 = x**4 - x**3 + x**2 - 7 * x + 7
g2 = 4 * x**3 - 3 * x**2 + 2 * x - 7

In [6]:
print("--- euclid_triang without using betas ---")
print()
print("Checking euclid_triang for f: {0} and g: {1}".format(f1, g1))
print(euclid_triang(f1, g1, x, use_beta=False))
print()
print("Checking euclid_triang for f: {0} and g: {1}".format(f2, g2))
print(euclid_triang(f2, g2, x, use_beta=False))

--- euclid_triang without using betas ---

Checking euclid_triang for f: x**3 + 3*x**2 - 7*x + 7 and g: 3*x**2 + 6*x - 7
[x**3 + 3*x**2 - 7*x + 7, 3*x**2 + 6*x - 7, 26208]

Checking euclid_triang for f: x**4 - x**3 + x**2 - 7*x + 7 and g: 4*x**3 - 3*x**2 + 2*x - 7
[x**4 - x**3 + x**2 - 7*x + 7, 4*x**3 - 3*x**2 + 2*x - 7, 23616*x - 33040, 35974400]


Παρατηρούμε ότι  οι συντελεστές των πολυωνύμων καταλήγουν να είναι αρκετά μεγάλοι.

Αυτό συμβαίνει διότι κατά την διαδικασία της τριγωνοποίησης, επειδή προσπαθούμε να έχουμε μόνο ακέραιους συντελεστές, η γραμμή που προσπαθούμε να απαλείψουμε πολλαπλασιάζεται με μεγάλα νούμερα.

Επιπλέον αυτή η διαδικασία συμβαίνει σε κάθε επανάληψη και για αυτό οδηγούμαστε σε τόσο μεγάλα νούμερα.

In [7]:
print("--- euclid_triang with betas ---")
print()
print("Checking euclid_triang for f: {0} and g: {1}".format(f1, g1))
print(euclid_triang(f1, g1, x))
print("euclid_amv(f, g, x) == euclid_triang(f, g, x): {0}".format(euclid_amv(f1, g1, x) == euclid_triang(f1, g1, x)))
print()
print("Checking euclid_triang for f: {0} and g: {1}".format(f2, g2))
print(euclid_triang(f2, g2, x))
print("euclid_amv(f, g, x) == euclid_triang(f, g, x): {0}".format(euclid_amv(f2, g2, x) == euclid_triang(f2, g2, x)))


--- euclid_triang with betas ---

Checking euclid_triang for f: x**3 + 3*x**2 - 7*x + 7 and g: 3*x**2 + 6*x - 7
[x**3 + 3*x**2 - 7*x + 7, 3*x**2 + 6*x - 7, 84 - 60*x, 2912]
euclid_amv(f, g, x) == euclid_triang(f, g, x): True

Checking euclid_triang for f: x**4 - x**3 + x**2 - 7*x + 7 and g: 4*x**3 - 3*x**2 + 2*x - 7
[x**4 - x**3 + x**2 - 7*x + 7, 4*x**3 - 3*x**2 + 2*x - 7, 5*x**2 - 82*x + 105, 1476*x - 2065, 5621]
euclid_amv(f, g, x) == euclid_triang(f, g, x): True


# Sylvester

## Sylvester1

## Sylvester2

# Το θεώρημα της Anna Johnson και η σημασία του

**1η Ερώτηση**

Παρατηρούμε ότι τα πρόσημα ακολουθούν το παρακάτω pattern $[++,--,++]$ το οποίο σημαίνει οποί είναι $+$ είναι ίδια ενώ στο $-$ αντίθετα.

**2η Ερώτηση**

Βλέπουμε ότι δεν ισχύει και για αυτά τα αποτελέσματα.

**3η Ερώτηση**

$$r_{k}^{(i)}=\frac{(-1)^{u_{i-1}}(-1)^{u_{i-}}...(-1)^{u_{1}}(-1)^{v_{i-1}}}{\rho_{i-1}^{p_{i-2}+1}\rho_{i-2}^{p_{i-2}+p_{i-1}}...\rho_{i-1}^{p_{1}+p_{2}}\rho_{0}^{p_{1}}}Det(i,k)$$

With
$$ u_{i-1}=1+2+...+p_{i-1}, v_{i-1}=p_{1}+p_{2}+...+p_{i-1}$$



\[Det(i,k)=
\begin{bmatrix}
    a_{0} & a_{1} & a_{2} & ... & . & . & ...   & a_{2v_{i-1}} & a_{2v_{i-1}+1+k} \\
    b_{0} & b_{1} & b_{2} & ... & . & . & ...   & b_{2v_{i-1}} & b_{2v_{i-1}+1+k} \\
    0     & a_{0} & a_{1} & ... & . & . & ...   & a_{2v_{i-1}-1} & a_{2v_{i-1}+k} \\
    0     & a_{0} & a_{1} & ... & . & . & ...   & a_{2v_{i-1}-1} & a_{2v_{i-1}+k} \\
    .     & .     & .     & ... & . & . & ...   & .              & . \\
    0     & 0     & 0     & ... & a_{0} & a_{1} & ... & a_{v_{i-1}} & a_{v_{i-1}+1+k} \\
    0     & 0     & 0     & ... & b_{0} & b_{1} & ... & a_{v_{i-1}} & a_{v_{i-1}+1+k} \\
\end{bmatrix}
\]

Πολυώνυμο από την sturm_sylv2 $-15x^{4}+3x^{2}-9$

Πολυώνυμο από sτην sturm_q $\frac{5}{9}x^{4}-\frac{1}{9}x^{2}+\frac{1}{3} \Rightarrow$

$-15x^{4}+3x^{2}-9 \Rightarrow -\frac{27}{1}\frac{5}{9}x^{4}+\frac{27}{1}\frac{1}{9}x^{2}-\frac{27}{1}\frac{1}{3} \Rightarrow-15x^{4}+3x^{2}-9$

# Ακολουθίες Sturm για την απομόνωση των πραγματικών ριζών πολυωνύμων με ακέραιους συντελεστες

In [8]:
def cauchy_upper_bound(f, x):
    n = degree(f, x)
    lc = LC(f, x)

    # extract coeffs and k in (k,c) format
    coeffs = []
    while not degree(f, x) < 1:
        t = LT(f)
        d = degree(t, x)
        coeffs.append((n - d, LC(t, x)))
        f = f - t
    else:
        coeffs.append((n, f))

    # if the leading coefficient is negative change all signs
    if lc < 0:
        coeffs = [(k, -c) for k, c in coeffs]
        lc = -lc

    # keep only the negative coefficients
    n_coeffs = [c for c in coeffs if c[1] < 0]

    lmbda = len(n_coeffs)
    if lmbda == 0:
        return 0

    # possible upper limits
    ub_candidates = []
    for k, c in n_coeffs:
        ub = (-lmbda * c) / lc
        ub = root(ub, k)
        ub_candidates.append(ub)

    return ceil((65 / 64) * max(ub_candidates))

In [9]:
# The function groupby from itertools create groups of consecutive items based on a condition
# for example if we have [1,1,-1,-1] it will create 2 groups [1,1] and [-1,-1]
# or for [1,-1,1,-1] 4 groups [1], [-1], [1], [-1] an in this way we can count the sign changes
def sign_var(num_list):
    num_list = [i for i in num_list if i != 0]
    return len(list(itertools.groupby(num_list, lambda i: i > 0))) - 1


def square_free_factors(f, x):
    r = gcd(f, diff(f, x))

    t = div(f, r)[0]
    sq_free_factors = []
    while degree(r, x) > 0:
        v = gcd(r, t)
        s = div(t, v)[0]
        sq_free_factors.append(s)
        r = div(r, v)[0]
        t = v
    else:
        sq_free_factors.append(t)

    p = 1
    for d, s in enumerate(sq_free_factors):
        p = p * (s**(d + 1))
    return p

In [10]:

def calculate_roots(f, x, prec=1e-7):
    f = square_free_factors(f, x)
    seq = sturm(f)

    upper_bound = cauchy_upper_bound(f, x)

    root_isolation_intervals = []
    intervals_to_be_processed = [[0, upper_bound]]

    while intervals_to_be_processed:
        interval = intervals_to_be_processed.pop()

        left_values = []
        right_values = []

        for s in seq:
            left_values.append(sign(s.subs({x: interval[0]})))

        for s in seq:
            right_values.append(sign(s.subs({x: interval[1]})))

        number_of_roots = sign_var(left_values) - sign_var(right_values)

        if number_of_roots == 0:
            continue
        elif number_of_roots == 1:
            root_isolation_intervals.append(interval)
        else:
            mp = sum(interval) / 2

            intervals_to_be_processed.append([interval[0], mp])
            intervals_to_be_processed.append([mp, interval[1]])

            if f.subs({x: mp}) == 0:
                root_isolation_intervals.append([mp, mp])

    refined_interval = []
    for interval in root_isolation_intervals:
        r = refine_root(f, interval[0], interval[1], eps=prec)
        refined_interval.append(r)

    if f.subs({x: 0}) == 0:
        refined_interval.append([0, 0])

    return refined_interval

In [11]:
# print the refined interval calcuted from calculate_roots
# digits only controls the print precision
def print_roots(f, x, digits=10):
    positive_roots = calculate_roots(f, x)
    negative_roots = calculate_roots(f.subs({x: -x}), x)

    positive_roots = [(Float(a, digits), Float(b, digits)) for a, b in positive_roots]
    negative_roots = [(Float(a, digits), Float(b, digits)) for a, b in negative_roots]
    print("Polynomial: {0}".format(f))
    print("Found: {0} positive roots at intervals: {1}".format(len(positive_roots), positive_roots))
    print("Found: {0} negative roots at intervals: {1}".format(len(negative_roots), negative_roots))

In [12]:
f1 = x**3 - 7 * x + 7
f2 = 32 * x**6 - 48 * x**4 + 18 * x**2 - 1
print_roots(f1, x)
print()
print_roots(f2, x)


Polynomial: x**3 - 7*x + 7
Found: 2 positive roots at intervals: [(1.692021435, 1.692021496), (1.356895850, 1.356895889)]
Found: 1 negative roots at intervals: [(3.048917318, 3.048917402)]

Polynomial: 32*x**6 - 48*x**4 + 18*x**2 - 1
Found: 3 positive roots at intervals: [(0.9659258221, 0.9659258267), (0.7071067499, 0.7071068124), (0.2588190184, 0.2588190549)]
Found: 3 negative roots at intervals: [(0.9659258221, 0.9659258267), (0.7071067499, 0.7071068124), (0.2588190184, 0.2588190549)]
